In [6]:
import transformers
import datasets
import nltk
import tokenizers
import regex
import colorama
import random

random.seed(16)

import segmentador
import tests
from config import *
print("Marker symbol (valid):", MARKER_VALID)
print("Marker symbol (noise):", MARKER_NOISE_START, MARKER_NOISE_END)

tests.load_registered_cases()

Marker symbol (valid): ✓
Marker symbol (noise): ❌s__ ❌e__
Loaded 0 test cases from './registered_test_case.csv'.


In [7]:
seg_model = segmentador.Segmenter(local_files_only=True)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model check

In [8]:
class DetectRecurrentNoise:
    RE_BARCODE = regex.compile(r"\*([\sA-Z0-9]+)\*")
    RE_PREAMBLE = regex.compile(
        r"^\s*(.{,60}?)[\s0-9]*" +
        "(?=C[aâ]mara\s*dos\s*deputados\s*(Proj|Req))",
        regex.IGNORECASE,
    )
    RE_BLANK_SPACES = regex.compile(r"\s+")
    
    @classmethod
    def _detect_barcode(cls, subpattern, text):
        pseudo_patterns = cls.RE_BARCODE.findall(text)
        
        if not pseudo_patterns:
            return text
        
        pseudo_patterns = sorted(set(pseudo_patterns))
        
        for pseudo_pattern in pseudo_patterns:
            pattern = list(cls.RE_BLANK_SPACES.sub("", pseudo_pattern))
            pattern.append("")
            pattern.insert(0, "")
            pattern = r"\s*".join(pattern)
            
            text = regex.sub(r"(\*" + pattern + r"\*" + pattern + ")", subpattern, text)
        
        return text
    
    @classmethod
    def _detect_preamble_noise(cls, subpattern, text):
        preamble = cls.RE_PREAMBLE.match(text)
    
        if not preamble or not preamble.group(1).strip():
            return text
        
        preamble_content = r"\s*".join(preamble.group(1).split(" "))
        preamble_content = preamble_content.replace(")", r"\)")
        preamble_content = preamble_content.replace("(", r"\(")
        text = regex.sub(r"(\s*" + preamble_content + r"[\s\d]*)", subpattern, text)
        return text
    
    @classmethod
    def sub(cls, subpattern: str, text: str, *args, **kwargs):
        text = cls._detect_barcode(subpattern, text)
        text = cls._detect_preamble_noise(subpattern, text)
        return text


RE_NOISE_BLOCKS = (
    regex.compile(
        r"((PL|PDL|PEC)\s*n[\.o\sº]*[\d\s]+/[\s\d]+)?+\s*"
        r"A\s*p\s*r\s*e\s*s\s*e\s*n\s*t\s*a\s*[cç]\s*[aã]\s*o\s*:"
        r"(\s*\d\s*){2}/(\s*\d\s*){2}/(\s*\d\s*){6}:(\s*\d){2}",
        regex.IGNORECASE | regex.MULTILINE,
    ),
    regex.compile(r"([0-9]{9,})"),
    regex.compile(r"(_{9,})"),
    regex.compile(r"(^[\s0-9]+|[\s0-9]+$)"),
)

QUOTES = r"”“\"'"

STANDARD_PREFIXES = (
    r"(?<=^|;(?:\s*e|\s*ou)?|[\.:\?]|\(\s*(?:NR|AC)\s*\)|" +
    f"[{QUOTES}]|" + 
    "|".join(reg.pattern for reg in RE_NOISE_BLOCKS) +
    f"|{MARKER_NOISE_START}.*?{MARKER_NOISE_END}\s*" +
    ")"
)
OPTIONAL_PREFIX_EXTENSIONS = (
    r"(?<=\s*(?:" +
    r"[0-9]{,2}" +
    r")\s*)"
)

VALID_ROMAN_NUM = r"M{0,3}(?:C[MD]|D?C{0,3})(?:X[CL]|L?X{0,3})(?:I?X|I?V|V?I{1,3})"

RE_PRE_BLOCKS = tuple(
    regex.compile(f"{STANDARD_PREFIXES}{OPTIONAL_PREFIX_EXTENSIONS}(?=\s*{pattern})", regex.IGNORECASE)
    for pattern in [
        r"§\s*[0-9]+",
        r"Art(?:igo)?s?\s*\.?\s*(?:[-–º0-9A-Z]+|\.{3}|[uú]nico)",
        r"(?:\s[A-Za-z]|[0-9]{1,2})\s*\)",
        r"(?:par[áa]grafo|§)\s*[úu]nico",
        r"(?:par[áa]grafo|§)\s*[0-9]{1,2}[\so0º]*:",
        r"(?:sub)?se[çc][ãa]o",
        r"\(?" + f"{VALID_ROMAN_NUM}" + r"\s*(?:[-–\)\.])",
        r"(?<!Art(?:igo)?\s?\.?\s?)\(?\s+[0-9]{1,2}[\s0oº]*(?:[-–\)]|\.(?![0-9]))",
        r"Sala\s*das?\s*(Sessões|comiss[aã]o|Reuni[oõ]es).{,200}$",
        r"Senado\s*Federal\s*,.{,200}$"
    ]
)

DEPT_EXTENSION_CORE = (
    r"(?:(?:Sra?|Senhora?)?[\s\.]*(?:Deputad[oa]|Dep\.)|" +
    r"(?:Sra?|Senhora?)[\s\.]*(?:Deputad[oa]|Dep\.)?|mesa\s*diretora)" +
    r"\s*"
)

DEPT_EXTENSION_A = (
    r"[^\(]{,100}\(\s*(?:D[oa])?\s*" +
    DEPT_EXTENSION_CORE +
    f"(?:[^{QUOTES}\)]" + r"{1,100})?\)"
)

DEPT_EXTENSION_B = (
    r".{,100}?D[oa]\s*" +
    DEPT_EXTENSION_CORE +
    f"(?:[^{QUOTES}]" + r"{1,100}" + f"?(?=[{QUOTES}]))?"
)

DEPT_EXTENSION = f"(?:{DEPT_EXTENSION_A}|{DEPT_EXTENSION_B})"
DATE_AND_ID = r"(?:(?:DE\s*)+?[\._0-9]+|N[\.\s]*[o0º](?:[^,]*?[,\.]+\s*(?:DE\s*)+?[\._0-9]+)?)"


RE_SPECIAL = (
    (regex.compile(
        r"(?<=^.{,250}?)(REQUERIMENTO\s*DE\s*INFORMA[cÇ](?:[oÕ]ES|[AÃ]O).{,50}?" +
        DATE_AND_ID +
        f"(?:{DEPT_EXTENSION})?" +
        r")\s*" +
        "(.{,600}?)(?=(?:Excelent[ií]ssim[oa])?\s*(?:Senhora?|Sra?)[\.\s*]Presidente)", regex.IGNORECASE),
     lambda symb, deb: f" {symb} {deb} " + r"\1" + f" {symb} {deb} " + r"\2" + f" {symb} {deb} "),
    (regex.compile(
        r"(?<=^.{,250}?)(REQUERIMENTO.{,25}?" +
        DATE_AND_ID +
        f"(?:{DEPT_EXTENSION})" +
        r")", regex.IGNORECASE),
     lambda symb, deb: f" {symb} {deb} " + r"\1" + f" {symb} {deb} "),
    (regex.compile(
        r"(INDICA[CÇ][AÃ]O.{,50}?" +
        DATE_AND_ID +
        f"(?:{DEPT_EXTENSION})" +
        r")\s*" +
        "(.{,600}?)(?=(?:Excelent[ií]ssim[oa])?\s*(?:Senhora?|Sra?)[\.\s*](?:Presidente|Ministr[oa]))", regex.IGNORECASE),
     lambda symb, deb: f" {symb} {deb} " + r"\1" + f" {symb} {deb} " + r"\2" + f" {symb} {deb} "),
    (regex.compile(
        r"(?<=^.{,250}?)((?:PROJETO\s*DE\s*)?RESOLU[CÇ][AÃ]O.{,50}?" + 
        DATE_AND_ID +
        f"(?:{DEPT_EXTENSION})?" +
        r")\s*" +
        r"(.{,600}?)((?:A\s*mesa\s*d)?A\s*C[âa]mara\s*dos\s*deputados[^\.]*?resolve\s*:)", regex.IGNORECASE),
     lambda symb, deb: f" {symb} {deb} " + r"\1" + f" {symb} {deb} " + r"\2" + f" {symb} {deb} " + r"\3" + f" {symb} {deb} "),
    (DetectRecurrentNoise, lambda symb_start, symb_end, deb: f" {symb_start} {deb} " + r"\1" + f" {symb_end} {deb} "),
)

RE_PRE_POST_BLOCKS = (
    regex.compile(r"(?<=^.{,250}?)(COMISS[AÃ]O\s*DE\s*CI[EÊ]NCIA[\sE]*TECNOLOGIA[\s,]*COMUNICA[CÇ][AÃ]O[\sE]*INFORM[AÁ]TICA)", regex.IGNORECASE),
    regex.compile(r"(O\s*Congresso\s*Nacional\s*decreta:)", regex.IGNORECASE),
    regex.compile(r"(A\s*C[aâ]mara\s*dos\s+deputados\s*decreta:)", regex.IGNORECASE),
    regex.compile(r"(?<=^.{,250}?)(Projeto\s*de\s*Lei\s*" + DEPT_EXTENSION + ")", regex.IGNORECASE),
    regex.compile(
        r"(?<=^.{,250}?)(Projeto\s*de\s*Decreto\s*Legislativo\s*" +
        f"(?:{DEPT_EXTENSION}|{DATE_AND_ID})" +
        ")",
        regex.IGNORECASE,
    ),
    regex.compile(
        r"(?<=^.{,250}?)(Proposta\s*de\s*emenda\s*(?:cons?titucional|[aàá]\s*constitui[çc][ãa]o).*?" +
        f"(?:{DEPT_EXTENSION})" +
        r")",
        regex.IGNORECASE,
    ),
    regex.compile(
        r"(cap[ií]tulo\s*" + f"{VALID_ROMAN_NUM}" +
        r"(?:[-–\sA-Za-zçàüáéíóúãõẽôâê0-9]|" + 
        f"{MARKER_NOISE_END}|{MARKER_NOISE_START}" +
        r")+?" +
        f"(?=(?:{MARKER_VALID}|Art)))",
        regex.IGNORECASE,
    ),
    regex.compile(
        r"(t[ií]tulo\s*" + f"{VALID_ROMAN_NUM}" +
        r"(?:[-–\sA-Za-zçàüáéíóúãõẽôâê0-9]|" + 
        f"{MARKER_NOISE_END}|{MARKER_NOISE_START}" +
        r")+?" +
        f"(?=(?:{MARKER_VALID}|cap[íi]tulo)))",
        regex.IGNORECASE,
    ),
)

def regex_legal_item_anymatch(text: str, debug: bool = False) -> str:
    aid = 0
    
    for i, reg in enumerate(RE_NOISE_BLOCKS, aid):
        debug_text = f"{i}_NOISE" if debug else ""
        text = reg.sub(f" {MARKER_NOISE_START} {debug_text} " + r"\1" + f" {MARKER_NOISE_END} {debug_text} ", text, concurrent=True)
    
    for i, (reg, fun) in enumerate(RE_SPECIAL, aid):
        debug_text = f"{i}_SPECIAL" if debug else ""
        try:
            pat = fun(MARKER_VALID, debug_text)
            
        except TypeError:
            pat = fun(MARKER_NOISE_START, MARKER_NOISE_END, debug_text)
            
        text = reg.sub(pat, text, concurrent=True)
        
    for i, reg in enumerate(RE_PRE_BLOCKS, aid):
        debug_text = f"{i}_PRE" if debug else ""
        text = reg.sub(f" {MARKER_VALID} {debug_text} ", text, concurrent=True)
        
    for i, reg in enumerate(RE_PRE_POST_BLOCKS, aid):
        debug_text = f"{i}_PRE_POS" if debug else ""
        text = reg.sub(f" {MARKER_VALID} {debug_text} " + r"\1" + f" {MARKER_VALID} {debug_text} ", text, concurrent=True)
        
    return text


auxaux="b ) todas as medicações prescritas com as dosagens utilizadas ; e c ) registro da quantidade de sangue recebida e dados que permitam identificar sua origem , sorologias efetuadas e prazo de validade ; XVI - ter assegurado , durante as consultas , internações , procedimentos diagnósticos , preventivos , cirúrgicos e terapêuticos e na satisfação de suas necessidades fisiológicas : 4 a ) a integridade física ; b ) a privacidade física ; c ) a individualidade ; d ) o respeito aos seus valores éticos e culturais ; e ) a confidencialidade de toda e qualquer informação pessoal ; e f ) a segurança do procedimento ; XVII - ser acompanhado , sempre que assim o desejar , nas consultas , exames e internações , por pessoa de sua livre escolha ; XVIII – ser acompanhada , se assim o desejar , por pessoa de sua livre escolha , no momento do parto e no pós-parto ; XIX – ter garantida a acessibilidade aos serviços , com o fim das barreiras arquitetônicas e de comunicabilidade , oferecendo condições de atendimento adequadas aos portadores de deficiências ou necessidades especiais ; XX - receber do profissional adequado , auxílio imediato e oportuno para a melhoria do conforto e bem-estar ; XXI - ter um local digno , respeitoso e adequado para o atendimento ; XXII - receber ou recusar assistência moral , psicológica , social ou religiosa ; XXIII - ser prévia e expressamente informado quando o tratamento proposto for experimental ou fizer parte de pesquisa , consentindo a participar de forma livre e esclarecida ; XXIV – ter informações relativas às ações de vigilância sanitária e epidemiológica , bem como sobre fatores de risco que afetem a saúde nos locais de moradia e trabalho ; XXV – ter acesso a anestesia em todas as situações em que esta for indicada , bem como a medicações e procedimentos que possam aliviar a dor e o sofrimento ; XXVI - recusar tratamentos dolorosos ou extraordinários para tentar prolongar a vida ; e 5 XXVII - optar pelo local de morte . § 1º - A criança , ao ser internada , terá em seu prontuário a relação das pessoas que poderão acompanhá-la integralmente durante o período de internação . § 2º - A atenção aos problemas de saúde mental realizar-se-á basicamente no âmbito comunitário , mediante práticas intersetoriais , assistência domiciliar e ambulatorial , sendo a internação utilizada como último recurso terapêutico , em ambiente o menos restritivo possível , objetivando a mais breve recuperação do paciente . Artigo 3º - É vedado aos serviços públicos de saúde e às entidades públicas e privadas conveniadas ou contratadas pelo Poder Público : I - realizar , proceder ou permitir qualquer forma de discriminação entre os usuários dos serviços de saúde ; II – prestar serviços ou ações de saúde discriminatórios , em termos de acesso ou qualidade dos procedimentos , entre usuários do SUS e beneficiários de planos , seguros , contratos ou convênios privados de saúde , próprios ou por eles intermediados ; e III - manter acessos diferenciados para os usuários do Sistema Único de Saúde e quaisquer outros usuários , em face de necessidades de atenção semelhantes . § 1º - O disposto no inciso III deste artigo compreende também as portas de entrada e saída , salas de estar , guichês , locais de agendamento , retirada de exames e locais de espera . Artigo 4º - Os serviços públicos de saúde e as entidades privadas , conveniadas ou contratadas pelo Poder Público , têm que garantir a todos os pacientes e usuários : I - a igualdade de acesso , em idênticas condições , a todo e qualquer procedimento para a assistência à saúde , médico ou não , inclusive administrativo , que se faça necessário e seja oferecido pela instituição ; e II - o atendimento equânime em relação à qualidade dos procedimentos referidos no inciso anterior . 6 Parágrafo único - O direito à igualdade de acesso a todos os serviços , exames , procedimentos e à sua qualidade , nos termos desta lei , é extensivo a autarquias , institutos , fundações , hospitais universitários e demais entidades públicas ou privadas , que recebam , a qualquer título , recursos do Sistema Único de Saúde . Artigo 5o . – As pessoas jurídicas de direito público e as de direito privado participantes ou não do SUS são responsáveis , objetivamente , pelos danos que seus agentes , nessa qualidade , causarem ao indivíduo ou à coletividade "
# _=preprocess_instance({"text": auxaux}, -1, True, debug=True)

In [9]:
df = datasets.load_dataset(
    "csv",
    data_files=["../data/content.txt"],
    header=None,
    names=["text"],
    cache_dir="../cache/datasets",
    nrows=6000,
)

RE_JUSTIFICATIVA = regex.compile(r"\s*(?:JUSTIFICATIVA|JUSTIFICA[CÇ][AÃ]O)")
RE_ANEXO = regex.compile(r"\s*ANEXO")

df = df.map(lambda item: {"text": RE_JUSTIFICATIVA.split(item["text"])[0]})
df = df.map(lambda item: {"text": RE_ANEXO.split(item["text"])[0]})
df = df.filter(lambda item: isinstance(item["text"], str) and 128 <= len(item["text"]) <= 600000)

def preprocess_instance(item, ind, print_preprocessed: bool = False, debug: bool = False):
    preprocessed_text = seg_model.preprocess_legal_text(item["text"])
    preprocessed_text = regex_legal_item_anymatch(preprocessed_text, debug=debug)
    tokens = nltk.tokenize.word_tokenize(preprocessed_text, language="portuguese")
    
    if print_preprocessed:
        print(preprocessed_text)
    
    labels = [0] * len(tokens)
    
    i = 0
    while i < len(tokens) - 1:
        if tokens[i] in SPECIAL_SYMBOLS:
            token = tokens.pop(i)
            labels.pop(i)
            labels[i] = SPECIAL_SYMBOLS[token]
            continue
            
        i += 1
    
    if labels[0] == SPECIAL_SYMBOLS[MARKER_VALID]:
        labels[0] = 0
    
    if tokens[-1] in SPECIAL_SYMBOLS:
        labels.pop()
        tokens.pop()

    ret = {
        "id": str(ind),
        "labels": labels,
        "tokens": tokens,
    }
    
    return ret

df = df.map(preprocess_instance, with_indices=True, num_proc=8, remove_columns="text")

Using custom data configuration default-13b3929bc6f4ae9b
Reusing dataset csv (../cache/datasets/csv/default-13b3929bc6f4ae9b/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at ../cache/datasets/csv/default-13b3929bc6f4ae9b/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-2731bf61aefd1b5e.arrow
Loading cached processed dataset at ../cache/datasets/csv/default-13b3929bc6f4ae9b/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-501a735807ce34be.arrow
Loading cached processed dataset at ../cache/datasets/csv/default-13b3929bc6f4ae9b/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-3b810910c03d94cd.arrow


In [10]:
tests.print_results(df, 3936, print_full_text=True)
tests.run_tests(df["train"]["labels"])
print("\n\n")

print(df.num_rows)

document_idx = 166
while tests.test_case_exists(document_idx):
    document_idx = random.randint(0, 1 + df["train"].num_rows)

print("Chosen id:", document_idx)
segment_count = tests.print_results(df, document_idx, print_full_text=True)
print("Is it correct? [y/N]:", end=" ")
inp = input()
if inp == "y":
    tests.update_test_case(document_idx, segment_count)
    print("Added to test cases.")

1 CÂMARA DOS DEPUTADOS Projeto de Lei nº , de 2003 ( Do Sr . Deputado Roberto Gouveia e Professor Luizinho ) Estabelece o Código Nacional de Direitos dos Usuários das Ações e dos Serviços de Saúde e dá outras providências O Congresso Nacional decreta : Artigo 1º - A prestação das ações e dos serviços de saúde aos usuários de qualquer natureza ou condição , em todo o território nacional , será universal e igualitária , nos termos da Lei 8080 de 1990 . Artigo 2º - São direitos dos usuários das ações e dos serviços de saúde , públicos e privados , em todo o país : I - ter atendimento digno , atencioso e respeitoso ; II – ter atendimento integral , livre de qualquer discriminação , restrição ou negação em função de : a ) idade ; b ) raça ; c ) gênero ; d ) orientação sexual ; e ) características genéticas ; f ) condições sociais ou econômicas ; g ) convicções culturais , políticas ou religiosas ; e h ) estado de saúde ou condição de portador de patologia , deficiência ou lesão preexistente




{'train': 5994}
Chosen id: 166
PROJETO DE LEI N.º , DE 2001 ( Do Sr . Deputado RONALDO VASCONCELLOS ) Altera a Lei nº 9.317 de 5 de dezembro de 1996 , que dispõe sobre o regime tributária das microempresas e das empresas de pequeno porte e institui o Sistema Integrado de Pagamento de Impostos e Contribuições das Microempresas e das Empresas de Pequeno Porte - SIMPLES e dá outras providências. . O Congresso Nacional decreta : Art . 1º . Fica acrescentado ao artigo 9º da Lei nº 9317 de 05 de dezembro de 1996 , o seguinte parágrafo : § - O disposto nos incisos V e XIII , não se aplica em relação às pessoas jurídicas enquadradas no SIMPLES na condição de Microempresa definida no inciso um do artigo 2º da Lei nº 9317 de 5 de dezembro de 1996 Art . 2º.Ficam acrescidos de cinqüenta por cento os percentuais referidos noa artigo 5º , inciso 1 , letra `` a '' , `` b '' e `` c '' da Lei nº 9317 de dezembro de 1996 . § Único - O produto da arrecadação proporcionada pelo disposto no Caput deste 

In [13]:
tests.dump_registered_cases()

Wrote 0 test cases at './registered_test_case.csv'.


In [ ]:
df["train"]["labels"]

In [ ]:
def tokenize_and_align_labels(examples):
    # source: https://huggingface.co/docs/transformers/custom_datasets#preprocess
    tokenized_inputs = seg_model.tokenizer(
        examples["tokens"],
        truncation=True,
        max_length=512,
        is_split_into_words=True,
    )

    labels = []
    
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs


df_tokenized = df.map(tokenize_and_align_labels, batched=True, num_proc=4)

In [ ]:
df_tokenized_train_eval_test = df_tokenized["train"].train_test_split(test_size=0.2, shuffle=True, seed=16)
df_tokenized_test_eval = df_tokenized_train_eval_test["test"].train_test_split(test_size=0.5, shuffle=False)
df_tokenized_split = datasets.DatasetDict({
    "train": df_tokenized_train_eval_test["train"],
    "eval": df_tokenized_test_eval["train"],
    "test": df_tokenized_test_eval["test"],
})
# df_tokenized_split.save_to_disk("../data/df_tokenized_split")
df_tokenized_split

In [ ]:
df_tokenized_split["train"].features

In [ ]:
print(df["train"]["labels"][49])